In [33]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import time

BATCH_SIZE = 128
NUM_EPOCHS = 10

In [34]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)
#get test data
test_x = Variable(torch.unsqueeze(test_dataset.data,dim=1)).type(torch.FloatTensor)[:2000]/255.
test_y = test_dataset.targets[:2000]


In [35]:
class SimpleNet(nn.Module):
# TODO:define model
    def __init__(self):
        super(SimpleNet,self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(    #(1,28,28)1是channel的维度，28*28为图片的长宽
                in_channels=1,#图片的层数，RGB=3,灰度=1
                out_channels=16,#filter的个数
                kernel_size=5,#filter的长宽
                stride=1,#每隔多少个移动
                padding=2,#图片补0.if stride = 1,padding =(kernel_size-1)/2 = (5-1)/2 = 2
            ),   #-->（16，28，28）
            nn.ReLU(),#-->（16，28，28）
            nn.MaxPool2d(kernel_size=2),#可以看成2*2的filter    #-->（16，14，14）kernel_size = 2,减小一半
        )
        self.conv2 = nn.Sequential(#-->（16，14，14）
            nn.Conv2d(16,32,5,1,2),#-->（32，14，14）
            nn.ReLU(),#-->（32，14，14）
            nn.MaxPool2d(2)#-->（32，7，7）

        )
        self.out = nn.Linear(32*7*7,10)#输出为10类

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)   #-->(batch,32,7,7)
        x = x.view(x.size(0),-1) #-->(batch,32*7*7)
        output = self.out(x)
        return output


    
model = SimpleNet()
print(model)
# TODO:define loss function and optimiter
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.01)

SimpleNet(
  (conv1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (out): Linear(in_features=1568, out_features=10, bias=True)
)


In [ ]:
# train and evaluate
for epoch in range(NUM_EPOCHS):
    for step,(images,labels) in enumerate(train_loader):
        # TODO:forward + backward + optimize
        b_x = Variable(images)
        b_y = Variable(labels)

        out = model(b_x)
        loss = criterion(out,b_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if step % 50 == 0:
            test_out = model(test_x)
            pred_y = torch.max(test_out,1)[1].data.squeeze()
            accuracy = float(sum(pred_y == test_y))/ float(sum(test_y == test_y))
            print('Epoch:',epoch,'|train loss:'+str(loss.item()),'|test accuracy:'+str(accuracy))

        
        
        
    # evaluate
    # TODO:calculate the accuracy using traning and testing dataset

Epoch: 0 |train loss:2.271793842315674 |test accuracy:0.223
Epoch: 0 |train loss:0.2260695993900299 |test accuracy:0.6845
Epoch: 0 |train loss:0.15258298814296722 |test accuracy:0.818
Epoch: 0 |train loss:0.1327541619539261 |test accuracy:0.81
Epoch: 0 |train loss:0.19820448756217957 |test accuracy:0.863
Epoch: 0 |train loss:0.08976157009601593 |test accuracy:0.9195
Epoch: 0 |train loss:0.07950440794229507 |test accuracy:0.914
Epoch: 0 |train loss:0.04449564591050148 |test accuracy:0.9295
Epoch: 0 |train loss:0.1385045200586319 |test accuracy:0.89
Epoch: 0 |train loss:0.19228792190551758 |test accuracy:0.8765
Epoch: 1 |train loss:0.02146744355559349 |test accuracy:0.875
Epoch: 1 |train loss:0.07584729790687561 |test accuracy:0.856
Epoch: 1 |train loss:0.20887112617492676 |test accuracy:0.902
Epoch: 1 |train loss:0.036342520266771317 |test accuracy:0.939
Epoch: 1 |train loss:0.05131036415696144 |test accuracy:0.8875
Epoch: 1 |train loss:0.020598676055669785 |test accuracy:0.901
Epoch: 1